In [2]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os 
import pandas as pd
import numpy as np

In [3]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [5]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
print (train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [7]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [9]:
from tqdm.notebook import tqdm

In [10]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, target_size=(48, 48), color_mode="grayscale")
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [11]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [12]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [13]:
x_train = train_features/255.0
x_test = test_features/255.0

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [16]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [17]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7) 

In [18]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

D:\python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
# --- Callbacks ---
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=6,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_emotion_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

model.fit(x= x_train,y = y_train, batch_size = 64, epochs = 50, validation_data = (x_test,y_test), callbacks=[early_stop, checkpoint]) 

Epoch 1/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 647ms/step - accuracy: 0.2441 - loss: 1.8351  
Epoch 1: val_accuracy improved from -inf to 0.25828, saving model to best_emotion_model.h5


451/451 ━━━━━━━━━━━━━━━━━━━━ 318s 689ms/step - accuracy: 0.2441 - loss: 1.8351 - val_accuracy: 0.2583 - val_loss: 1.8086
Epoch 2/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 612ms/step - accuracy: 0.2483 - loss: 1.8076  
Epoch 2: val_accuracy improved from 0.25828 to 0.25927, saving model to best_emotion_model.h5


451/451 ━━━━━━━━━━━━━━━━━━━━ 293s 650ms/step - accuracy: 0.2483 - loss: 1.8075 - val_accuracy: 0.2593 - val_loss: 1.7848
Epoch 3/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 597ms/step - accuracy: 0.2776 - loss: 1.7529  
Epoch 3: val_accuracy improved from 0.25927 to 0.35451, saving model to best_emotion_model.h5


451/451 ━━━━━━━━━━━━━━━━━━━━ 286s 635ms/step - accuracy: 0.2776 - loss: 1.7529 - val_accuracy: 0.3545 - val_loss: 1.6547
Epoch 4/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 597ms/step - accuracy: 0.3368 - loss: 1.6532  
Epoch 4: val_accuracy improved from 0.35451 to 0.41438, saving model to best_emotion_model.h5


451/451 ━━━━━━━━━━━━━━━━━━━━ 286s 635ms/step - accuracy: 0.3368 - loss: 1.6532 - val_accuracy: 0.4144 - val_loss: 1.5092
Epoch 5/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 611ms/step - accuracy: 0.3865 - loss: 1.5652  
Epoch 5: val_accuracy improved from 0.41438 to 0.45684, saving model to best_emotion_model.h5


...

451/451 ━━━━━━━━━━━━━━━━━━━━ 288s 639ms/step - accuracy: 0.5436 - loss: 1.2130 - val_accuracy: 0.5811 - val_loss: 1.1286
Epoch 32/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 597ms/step - accuracy: 0.5522 - loss: 1.1861  
Epoch 32: val_accuracy did not improve from 0.58109
451/451 ━━━━━━━━━━━━━━━━━━━━ 286s 634ms/step - accuracy: 0.5522 - loss: 1.1862 - val_accuracy: 0.5735 - val_loss: 1.1331
Epoch 33/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 596ms/step - accuracy: 0.5443 - loss: 1.1930  
Epoch 33: val_accuracy did not improve from 0.58109
451/451 ━━━━━━━━━━━━━━━━━━━━ 286s 634ms/step - accuracy: 0.5443 - loss: 1.1931 - val_accuracy: 0.5725 - val_loss: 1.1422
Epoch 34/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 596ms/step - accuracy: 0.5466 - loss: 1.1989  
Epoch 34: val_accuracy improved from 0.58109 to 0.58279, saving model to best_emotion_model.h5


451/451 ━━━━━━━━━━━━━━━━━━━━ 286s 635ms/step - accuracy: 0.5466 - loss: 1.1989 - val_accuracy: 0.5828 - val_loss: 1.1159
Epoch 35/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 596ms/step - accuracy: 0.5524 - loss: 1.1853  
Epoch 35: val_accuracy did not improve from 0.58279
451/451 ━━━━━━━━━━━━━━━━━━━━ 286s 634ms/step - accuracy: 0.5524 - loss: 1.1853 - val_accuracy: 0.5767 - val_loss: 1.1263
Epoch 36/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 598ms/step - accuracy: 0.5528 - loss: 1.1861  
Epoch 36: val_accuracy improved from 0.58279 to 0.58322, saving model to best_emotion_model.h5


451/451 ━━━━━━━━━━━━━━━━━━━━ 287s 637ms/step - accuracy: 0.5528 - loss: 1.1861 - val_accuracy: 0.5832 - val_loss: 1.1166
Epoch 37/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 597ms/step - accuracy: 0.5513 - loss: 1.1811  
Epoch 37: val_accuracy did not improve from 0.58322
451/451 ━━━━━━━━━━━━━━━━━━━━ 286s 635ms/step - accuracy: 0.5513 - loss: 1.1811 - val_accuracy: 0.5778 - val_loss: 1.1260
Epoch 38/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 601ms/step - accuracy: 0.5605 - loss: 1.1692  
Epoch 38: val_accuracy improved from 0.58322 to 0.59043, saving model to best_emotion_model.h5


451/451 ━━━━━━━━━━━━━━━━━━━━ 288s 639ms/step - accuracy: 0.5605 - loss: 1.1692 - val_accuracy: 0.5904 - val_loss: 1.1057
Epoch 39/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 602ms/step - accuracy: 0.5592 - loss: 1.1602  
Epoch 39: val_accuracy did not improve from 0.59043
451/451 ━━━━━━━━━━━━━━━━━━━━ 289s 641ms/step - accuracy: 0.5592 - loss: 1.1602 - val_accuracy: 0.5894 - val_loss: 1.0948
Epoch 40/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 601ms/step - accuracy: 0.5594 - loss: 1.1698  
Epoch 40: val_accuracy did not improve from 0.59043
451/451 ━━━━━━━━━━━━━━━━━━━━ 288s 640ms/step - accuracy: 0.5594 - loss: 1.1698 - val_accuracy: 0.5886 - val_loss: 1.1027
Epoch 41/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 613ms/step - accuracy: 0.5570 - loss: 1.1605  
Epoch 41: val_accuracy did not improve from 0.59043
451/451 ━━━━━━━━━━━━━━━━━━━━ 300s 665ms/step - accuracy: 0.5571 - loss: 1.1605 - val_accuracy: 0.5868 - val_loss: 1.1051
Epoch 42/50
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 697ms/step - accuracy: 0.5652 - loss: 1.144

451/451 ━━━━━━━━━━━━━━━━━━━━ 343s 761ms/step - accuracy: 0.5688 - loss: 1.1402 - val_accuracy: 0.5920 - val_loss: 1.0986
Epoch 45/50
221/451 ━━━━━━━━━━━━━━━━━━━━ 2:53 756ms/step - accuracy: 0.5692 - loss: 1.1388

In [36]:
model_json = model.to_json()
with open("emotiondetectorV2.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetectorV2.keras")